In [70]:
import datetime

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [71]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# Extract

In [72]:
df_trans = pd.read_sql_table('trans_servicio', etl_conn)
dim_persona = pd.read_sql_table('dim_persona', etl_conn)
dim_medico = pd.read_sql_table('dim_medico', etl_conn)
dim_servicio = pd.read_sql_table('dim_servicio', etl_conn)
dim_ips = pd.read_sql_table('dim_ips', etl_conn)
dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)

# Transformations

In [73]:
hecho_atencion = pd.merge(df_trans,dim_fecha[['date','key_dim_fecha']],left_on='fecha_atencion',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_dim_fecha':'key_fecha_atencion','id_medico':'cedula','id_usuario':'numero_identificacion'}, inplace=True)
hecho_atencion = pd.merge(hecho_atencion,dim_fecha[['date','key_dim_fecha']],left_on='fecha_solicitud',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_dim_fecha':'key_fecha_solicitud'}, inplace=True)

hecho_atencion.head()

,key_trans_servicio,codigo_servicio,numero_identificacion,cedula,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,tipo_servicio,fecha_hora_atencion,fecha_hora_solicitud,saved,key_fecha_atencion,key_fecha_solicitud
0,0,808809,1705225684180,272113543,2007-03-19,28800000000000,2007-03-19,30600000000000,hospitalizacion,2007-03-19 08:30:00,2007-03-19 08:00:00,2024-04-22,807,807
1,1,808811,1705225699140,81113543,2006-12-25,28800000000000,2006-12-25,30600000000000,hospitalizacion,2006-12-25 08:30:00,2006-12-25 08:00:00,2024-04-22,723,723
2,2,808813,1705225788780,166113543,2007-12-16,28800000000000,2007-12-16,30480000000000,hospitalizacion,2007-12-16 08:28:00,2007-12-16 08:00:00,2024-04-22,1079,1079
3,3,808815,1705225786530,228113543,2007-01-07,28800000000000,2007-01-07,30840000000000,hospitalizacion,2007-01-07 08:34:00,2007-01-07 08:00:00,2024-04-22,736,736
4,4,808817,1705225880540,172113543,2006-10-29,28800000000000,2006-10-29,30360000000000,hospitalizacion,2006-10-29 08:26:00,2006-10-29 08:00:00,2024-04-22,666,666


In [74]:
hecho_atencion= hecho_atencion.merge(dim_persona[['key_dim_persona','numero_identificacion']])
hecho_atencion.drop(columns=['numero_identificacion'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_medico[['key_dim_medico','cedula','id_ips']])
hecho_atencion.drop(columns=['cedula'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_ips[['key_dim_ips','id_ips']])
hecho_atencion.drop(columns=['id_ips'], inplace=True)
hecho_atencion.head()

,key_trans_servicio,codigo_servicio,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,tipo_servicio,fecha_hora_atencion,fecha_hora_solicitud,saved,key_fecha_atencion,key_fecha_solicitud,key_dim_persona,key_dim_medico,key_dim_ips
0,0,808809,2007-03-19,28800000000000,2007-03-19,30600000000000,hospitalizacion,2007-03-19 08:30:00,2007-03-19 08:00:00,2024-04-22,807,807,2841,271,23
1,1,808811,2006-12-25,28800000000000,2006-12-25,30600000000000,hospitalizacion,2006-12-25 08:30:00,2006-12-25 08:00:00,2024-04-22,723,723,2846,80,5
2,2,808813,2007-12-16,28800000000000,2007-12-16,30480000000000,hospitalizacion,2007-12-16 08:28:00,2007-12-16 08:00:00,2024-04-22,1079,1079,2899,165,13
3,3,808815,2007-01-07,28800000000000,2007-01-07,30840000000000,hospitalizacion,2007-01-07 08:34:00,2007-01-07 08:00:00,2024-04-22,736,736,2897,227,17
4,4,808817,2006-10-29,28800000000000,2006-10-29,30360000000000,hospitalizacion,2006-10-29 08:26:00,2006-10-29 08:00:00,2024-04-22,666,666,2957,171,14


In [75]:
hecho_atencion= hecho_atencion.merge(dim_servicio[['name','key_dim_servicio']],left_on='tipo_servicio',right_on='name')
hecho_atencion.drop(columns=['name','tipo_servicio'], inplace=True)
hecho_atencion.head()

,key_trans_servicio,codigo_servicio,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,fecha_hora_atencion,fecha_hora_solicitud,saved,key_fecha_atencion,key_fecha_solicitud,key_dim_persona,key_dim_medico,key_dim_ips,key_dim_servicio
0,0,808809,2007-03-19,28800000000000,2007-03-19,30600000000000,2007-03-19 08:30:00,2007-03-19 08:00:00,2024-04-22,807,807,2841,271,23,1
1,1,808811,2006-12-25,28800000000000,2006-12-25,30600000000000,2006-12-25 08:30:00,2006-12-25 08:00:00,2024-04-22,723,723,2846,80,5,1
2,2,808813,2007-12-16,28800000000000,2007-12-16,30480000000000,2007-12-16 08:28:00,2007-12-16 08:00:00,2024-04-22,1079,1079,2899,165,13,1
3,3,808815,2007-01-07,28800000000000,2007-01-07,30840000000000,2007-01-07 08:34:00,2007-01-07 08:00:00,2024-04-22,736,736,2897,227,17,1
4,4,808817,2006-10-29,28800000000000,2006-10-29,30360000000000,2006-10-29 08:26:00,2006-10-29 08:00:00,2024-04-22,666,666,2957,171,14,1


# tiempo de espera

In [76]:
hecho_atencion['tiempo_espera'] = hecho_atencion['fecha_hora_atencion'] - hecho_atencion['fecha_hora_solicitud']
hecho_atencion['tiempo_espera_dias'] = hecho_atencion['tiempo_espera'].dt.days
hecho_atencion['tiempo_espera_minutos'] = hecho_atencion['tiempo_espera'].dt.seconds // 60 
hecho_atencion['tiempo_espera_horas'] = hecho_atencion['tiempo_espera'].dt.seconds // (60 * 60)
hecho_atencion['tiempo_espera_segundos'] = hecho_atencion['tiempo_espera'].dt.seconds

hecho_atencion.drop(columns=['tiempo_espera','fecha_atencion','fecha_solicitud','hora_solicitud','hora_atencion','fecha_hora_solicitud','fecha_hora_atencion','codigo_servicio'], inplace=True)
hecho_atencion.head()


,key_trans_servicio,saved,key_fecha_atencion,key_fecha_solicitud,key_dim_persona,key_dim_medico,key_dim_ips,key_dim_servicio,tiempo_espera_dias,tiempo_espera_minutos,tiempo_espera_horas,tiempo_espera_segundos
0,0,2024-04-22,807,807,2841,271,23,1,0,30,0,1800
1,1,2024-04-22,723,723,2846,80,5,1,0,30,0,1800
2,2,2024-04-22,1079,1079,2899,165,13,1,0,28,0,1680
3,3,2024-04-22,736,736,2897,227,17,1,0,34,0,2040
4,4,2024-04-22,666,666,2957,171,14,1,0,26,0,1560


In [77]:
from datetime import date

hecho_atencion['saved'] = date.today()

# load

In [78]:
hecho_atencion.to_sql('hecho_atencion',etl_conn,if_exists='replace',index_label='key_hecho_atencion')

465